Langchain을 활용한 모델 사용, 비용 모니터잉 및 캐싱 전략
  - GPT-4o-mini  GPT-3.5-Turbo 비용이 60% 저렴
  - Langchain V3.0x 부터 openai 가 별도 패키지로 분리 필요 패키지를 설치 langchain-openai 필요
  - 토큰 사용량 추적, 캐싱을 위한 langchain-community 도 별도 설치
  - 환경 변수 관리 패키지 python-dotenv

In [4]:
%pip install langchain-openai langchain-community python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 환경변수 로드
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7) # dotenv 를 하면 자동으로 OPENAI_API_KEY를 참조
# LLM에 질의하기
prompt = "Langchain에 대해 한 문장으로 설명해줘" 
result = llm.invoke(prompt)
result.content

'Langchain은 자연어 처리(NLP) 애플리케이션을 개발하기 위한 프레임워크로, 다양한 언어 모델과 데이터 소스를 연결하여 복잡한 언어 작업을 수행할 수 있게 해줍니다.'

In [7]:
# 사용량
result.usage_metadata

{'input_tokens': 18,
 'output_tokens': 50,
 'total_tokens': 68,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [9]:
# 콜백함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘?')
    print("응답1",res1.content[:10], '...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법을 알려줘')
    print('응답2',res2.content[:10], '...')
# 누적 토큰 사용량 출력 콜백 cb에는 블록 내 전체 토큰 사용량이 누적
# 총 토큰 사용량
print(f"총 토큰 사용량: {cb.total_tokens}")
# 프롬프트 토큰수
print(f"프롬프트 토큰수: {cb.prompt_tokens}")
# 응답 토큰수
print(f"응답 토큰수: {cb.completion_tokens}")
# 비용 계산
print(f"비용: ${cb.total_cost}")

응답1 죄송하지만, 실시간 ...
응답2 랭체인(LangCh ...
총 토큰 사용량: 779
프롬프트 토큰수: 39
응답 토큰수: 740
비용: $0.0004498499999999999


## Langchain의 LLM 응답캐싱 (In-memort Cache, SQLiteCache)
하는 이유 : 동일한 질문은 저장해뒀다가 응답에 사용

In [11]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCache를 글로벌 캐시로 설정
set_llm_cache(InMemoryCache())

In [18]:
# 캐시 사용 전후를 비교, 같은 질문을 두번 호출
query = "재미있는 유머 하나 알려줘"
# 첫번째 호출(캐시에 없으면 api 호출 발생)
result1 = llm.invoke(query)
print("응답1:", result1.content)
print('*'*100)
# 두번째 호출(동일한 query. 캐시를 확인하고 동일 질문이면 api 미호출)
query = "재미있는 유머 하나 알려줘"
result2 = llm.invoke(query)
print("응답2:", result2.content)


응답1: 물론이죠! 다음은 짧은 유머입니다:

왜 컴퓨터는 바다에 가지 않나요?

바다에 가면 "버그"가 생길까 봐요! 

어떻게 생각하세요? 😊
****************************************************************************************************
응답2: 물론이죠! 다음은 짧은 유머입니다:

왜 컴퓨터는 바다에 가지 않나요?

바다에 가면 "버그"가 생길까 봐요! 

어떻게 생각하세요? 😊


In [ ]:
# 실행시간 측정
import time
# 첫번째 호출 시간
query = "점심 메뉴 추천해줘"
start = time.time() ; llm.invoke(query) ;end = time.time()
print(f"첫번째 호출 시간: {end - start}초")

# 실행시간 측정
# 두번째 호출 시간
start = time.time() ; llm.invoke(query) ;end = time.time()
print(f"두번째 호출 시간: {end - start}초")


# 첫번째 호출 시간: 1.6040949821472168초
# 두번째 호출 시간: 0.00099945068359375초
# 캐시메모리에서 확인했으므로 시간 단축 
# 같은 질문에 대해서 정답 일관성이 높아지고, 시간 단축 효과가 있다.


첫번째 호출 시간: 1.6040949821472168초
두번째 호출 시간: 0.00099945068359375초


# SQLite 캐시(디스크기반 캐시)

In [32]:
# SQLiteCache를 글로벌 캐시로 설정
import os
from langchain_community.cache import SQLiteCache
# 기존 캐시 DB 파일이 있다면 삭제(.langchain.db 초기화)
# if os.path.exists('.langchain.db'): # True False 결정 함수
#     os.remove('.langchain.db') # 있으면 삭제

# SQLiteCache를 글로벌 캐시로 설정(지정한 경로의 DB 파일을 생성/사용)
set_llm_cache(SQLiteCache(database_path='.langchain2.db')) # langchain.db 있으면 사용 없으면 생성


In [34]:
import time
# 동일한 query를 두번 호출해서 결과와 시간을 비교
query = "썰렁한 유머 하나 알려줘"
# 첫번째 호출(캐시에 없으면 api 호출 발생)
start = time.time() ; result1 = llm.invoke(query) ; end = time.time()
print(f'첫번째 호출 시간: {end - start}초')
print("응답1:", result1.content)

# 두번째 호출(동일한 query. 캐시를 확인하고 동일 질문이면 api 미호출)
start = time.time() ; result2 = llm.invoke(query) ; end = time.time()         
print(f'두번째 호출 시간: {end - start}초')
print("응답2:", result2.content)


# 첫번째 호출 시간: 1.6040949821472168초
# 두번째 호출 시간: 0.0020258426666259766초

첫번째 호출 시간: 0.0019998550415039062초
응답1: 왜 바다가 짠지 아세요? 

소금이 너무 많아서요! 😄
두번째 호출 시간: 0.0014193058013916016초
응답2: 왜 바다가 짠지 아세요? 

소금이 너무 많아서요! 😄
